<a href="https://colab.research.google.com/github/LukasEder1/CKE-Evaluation/blob/master/CreateKeywordCollections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/LukasEder1/CKE-Evaluation.git"
%cd "/content/CKE-Evaluation"

Cloning into 'CKE-Evaluation'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 84 (delta 38), reused 22 (delta 7), pack-reused 0
Unpacking objects: 100% (84/84), 910.37 KiB | 2.66 MiB/s, done.
/content/CKE-Evaluation


In [2]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-6hj1799l
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-6hj1799l
  Resolved https://github.com/boudinfl/pke.git to commit f2d4f5d2252c64d23defccd32fdac8809cfd7ce0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# Keyword Collection Creation

In [3]:
from contrastive_keyword_extraction import contrastive_extraction, final_score
import sqlite3
import pandas as pd
from baselines import *
from tqdm import tqdm
import string
import pickle
import sentence_comparision
import sentence_importance
import summary
import utilities
import keyword_extraction
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
with open("policy_data.pkl", "rb") as file:
  policies = pickle.load(file)

with open("article_data.pkl", "rb") as file:
  articles = pickle.load(file)

In [8]:
def create_collection(sites, 
                      ke_extractor = keyword_extraction.extract_yake, 
                      num_keywords=10,
                      max_ngram=2, 
                      sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                      importance_estimator = sentence_importance.text_rank_importance,
                      name_prefix="",
                      make_data_persistent=False, 
                      path="dataframes",
                      threshold=0.6,
                      stopwords=nltk.corpus.stopwords.words("english"),
                      combinator=utilities.alpha_combination,
                      gamma = 0.5,
                      num_splits=1,
                      matching_model="all-MiniLM-L6-v2"):
    
    for i in tqdm(sites.keys()):

        documents = sites[i]
        
        # run CKE-pipeline
        # Extract Keywords, and Matched sentences
        keywords, matched_dict, changed_indices, additions, deletions, new_indices, ranking, removed, matched_indices, unified_delitions = contrastive_extraction(documents, 
                                                                            max_ngram=max_ngram,
                                                                            min_ngram=1, 
                                                                            show_changes=False, 
                                                                            symbols_to_remove=string.punctuation,
                                                                            importance_estimator=importance_estimator,
                                                                            match_sentences=sentence_matcher,
                                                                            threshold=threshold,
                                                                            extra_stopwords=stopwords,
                                                                            top_k=int(num_splits),
                                                                            combinator=combinator,
                                                                            alpha_gamma=gamma,
                                                                            matching_model=matching_model)
        
        

    
        # create itermediate
        kws, scores = extract_from_dict(keywords)
        
        pipeline_frame = pd.DataFrame({'keyword': kws, 'score': scores})
        
        #extractor = lambda x: ke_extractor(x, max_ngram_size=max_ngram, numOfKeywords=num_keywords)
        
        # create CKE on the specified baseline
        baseline_keywords = baseline_diff_content(additions, unified_delitions, ke_extractor, num_keywords, max_ngram)
        
        baseline_kws, baseline_scores = extract_from_tuple_list(baseline_keywords)
        
        baseline_frame1 = pd.DataFrame({'keyword': baseline_kws, 'score': baseline_scores})
        
        
        # create CKE for baseline method 2
        baseline_keywords2 = baseline_keywords_in_diff(documents, ke_extractor, additions, deletions, candidates=50, max_ngram=max_ngram)
        
        baseline_kws2, baseline_scores2 = extract_from_dict(baseline_keywords2)
        
        baseline_frame2 = pd.DataFrame({'keyword': baseline_kws2, 'score': baseline_scores2})
        
        # Baseline 3
        baseline_keywords3 = baseline3(documents, additions, unified_delitions, max_ngram)
        
        baseline_kws3, baseline_scores3 = extract_from_dict(baseline_keywords3)
        
        baseline_frame3 = pd.DataFrame({'keyword': baseline_kws3, 'score': baseline_scores3})

        # Baseline 4
        baseline_keywords4 = baseline4(documents, max_ngram, stopwords)
        
        baseline_kws4, baseline_scores4 = extract_from_dict(baseline_keywords4)
        
        baseline_frame4 = pd.DataFrame({'keyword': baseline_kws4, 'score': baseline_scores4})

        # decide, wether to actually save the data
        if make_data_persistent:
            
            pipeline_frame.to_csv(f"{path}/{name_prefix}_pipeline_keywords_{i}.csv", index=False)
            
            baseline_frame1.to_csv(f"{path}/{name_prefix}_baseline1_keywords_{i}.csv", index=False)
            
            baseline_frame2.to_csv(f"{path}/{name_prefix}_baseline2_keywords_{i}.csv", index=False)
            
            baseline_frame3.to_csv(f"{path}/{name_prefix}_baseline3_keywords_{i}.csv", index=False)
            
            baseline_frame4.to_csv(f"{path}/{name_prefix}_baseline4_keywords_{i}.csv", index=False)
    
    return pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4

In [9]:
a = create_collection(articles, 
                  ke_extractor = keyword_extraction.extract_yake, 
                  num_keywords=15, # relevant for baselines
                  max_ngram=2, 
                  sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                  importance_estimator = sentence_importance.text_rank_importance,
                  name_prefix="standard",
                  make_data_persistent=True, 
                  path="Dataframes",
                  threshold=0.6,
                  stopwords=nltk.corpus.stopwords.words("english"),
                  combinator=utilities.alpha_combination,
                  gamma = 0.5,
                  num_splits=1,
                  matching_model ="msmarco-distilbert-base-v4")

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [05:35<00:00,  3.35s/it]


In [18]:
from itertools import product


def cartesian_product(params):
    
    # gett all possible combinations
    return list(product(*params.values()))



# Parameters to Use

In [27]:
parameters = {"matcher": [sentence_comparision.match_sentences_semantic_search,
                          sentence_comparision.match_sentences_tfidf_weighted],
              
             "ie": [sentence_importance.text_rank_importance],
              
             "threshold": [0.5, 0.6, 0.7, 0.8]
              
             }


In [28]:
combinations = cartesian_product(parameters)
num_of_combinations = len(combinations)

In [29]:
def create_cartesian_collection(params, 
                                sites, 
                                baseline_ke_extractor = keyword_extraction.extract_yake, 
                                num_keywords=10, 
                                max_ngram=2, 
                                make_data_persistent=False,
                                file_prefix = "combination",
                                path="Dataframes",
                                stopwords=nltk.corpus.stopwords.words("english"),
                                combinator=utilities.alpha_combination,
                                gamma = 0.5,
                                num_splits=1,
                                matching_model="all-MiniLM-L6-v2"):
            
        
    combinations = cartesian_product(params)
    
    number_of_combinations = len(combinations)
    
    count = 0
    
    for combination in combinations:
        
        matcher, ie, threshold = combination
        
        print(f"Contrastive Keyword Extraction pipeline is being ran with combination {count}:")
        
        pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4 = create_collection(sites = sites, 
                                                                  ke_extractor = baseline_ke_extractor, 
                                                                  num_keywords = num_keywords, 
                                                                  max_ngram = max_ngram, 
                                                                  sentence_matcher = matcher,
                                                                  importance_estimator = ie,
                                                                  name_prefix=f"{file_prefix}_{count}",
                                                                  make_data_persistent=make_data_persistent,
                                                                  path=path,
                                                                  threshold=threshold,
                                                                  combinator=combinator,
                                                                  gamma = gamma,
                                                                  num_splits=num_splits,
                                                                  matching_model = matching_model)




        count += 1
        

In [30]:
create_cartesian_collection(parameters, 
                            articles, 
                            baseline_ke_extractor = keyword_extraction.extract_yake, 
                            num_keywords=15, 
                            max_ngram=2, 
                            make_data_persistent=True,
                            file_prefix = "combination",
                            path="Dataframes",
                            stopwords=nltk.corpus.stopwords.words("english"),
                            combinator=utilities.alpha_combination,
                            gamma = 0.5,
                            num_splits=1,
                            matching_model="all-MiniLM-L6-v2")

Contrastive Keyword Extraction pipeline is being ran with combination 0:


100%|██████████| 100/100 [04:16<00:00,  2.57s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 1:


100%|██████████| 100/100 [04:12<00:00,  2.53s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 2:


...

100%|██████████| 100/100 [03:36<00:00,  2.16s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 6:


100%|██████████| 100/100 [03:33<00:00,  2.14s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 7:


100%|██████████| 100/100 [03:35<00:00,  2.16s/it]


# Evaluate Results


## Compare against baselines

In [ ]:
summary.extensive_summary(list(policies.keys()),
                          show_results=True, 
                          k=15,
                          name_a = f"standard_pipeline_keywords", 
                          name_b = f"standard_baseline1_keywords",
                          save_prefix=f"standard_", 
                          path='Dataframes')


,Site,F1,Precision,Recall,IoU,#overlaps
0,90536,0.0,0.0,0.0,0.0,0
1,90344,0.0,0.0,0.0,0.0,0
2,98640,0.0,0.0,0.0,0.0,0
3,98585,0.0,0.0,0.0,0.0,0
4,99880,0.0,0.0,0.0,0.0,0
5,108079,0.0,0.0,0.0,0.0,0
6,90555,0.0,0.0,0.0,0.0,0
7,90545,0.0,0.0,0.0,0.0,0
8,98553,0.0,0.0,0.0,0.0,0
9,98572,0.0,0.0,0.0,0.0,0


## Manually Annotated Keywords

In [10]:
news = {

  # AP - Magazine 2-grams

    16532: ["Toys R", "R us", "toy makers", "Hong Kong","Asian sourcing", "sourcing giant"
         ," giant li", "Fung"],#meta: Toys R us

    7335: ["nuclear weapons", "united states", "north korea", "Yoon said", "Park Geun-hye",
            "missile provocation", "last week", "Hwasong-12 missile"],

    16625: ["50 million", "million users", "trump campaign", "social media",
        "facebook profiles", "Russian election", " Texas Republican", "$15 million",
        "Mueller's investigation"],

    3694: ["filmmaker Jason", "Jason Pollock", "department justice", "Pollock said",
           "Monday night", "deceptive editing"],

    7409: ["Puerto ricans", "statehood", "lost loved", "loved ones",
        "military veterans"], #meta: lost loved ones

    1525: ["white house", "TRUMP ADMINISTRATION", "palestinian officals",
        "palestinian leadership", "statehood", "Europe's view",
        ""],#meta: white house

    16699: ["prime minister", "foreign leaders", "bilateral meeting", "congratulatory telegrams",
        "German Chancellor", "Angela Merkel", "Xi Jinping", "Emmanuel Macron"],


    8370: ["removed 92", "92 million", "million videos", "share encrypted", "encrypted information",
     "encrypted data", "tech companies", "law enforcement"],

    15397: ["31, 2018", "united states", "Quarterly progress"],

    17354: ["Researcher Autumn", "Autumn Stroup", "city regulations", "Transgender activists",
        "public accommodations", "accommodations laws", "tracking city"],

    877: ["Supreme Court", "new Turkish", "flight engineers", "eight pilots",
     "expected within", "within days", "requested asylum"],

    15494: ["preferred description", " call the law", "betrayal middle-class", "middleclass households",
     "tax law"], #meta: tax law

    17769: ["north korea", "wrongful death", "kim jong", "jong un"],

    4138: ["foreign aid", "medical research", "proposed cuts"],

    8811: ["Borg Institute", "many women", "technical worlds", "cite Starbucks",
        "flying high"],

    #
    16018: ["Systemic poverty", "five decades", "released Tuesday", "made public",
        "public tuesday", "mortgage lending", "mortgage leaders", "new report"],

    4140: ["temporary appointments", "Justice Ruth", "Ruth Bader", "Bader Ginsburg",
        "never objected", "objection from Congress", "Supreme Court"],

    #9656: [""],
    16239: ["AP league","league player", "Big 12", "remaining power", "power conferences",
        "Georgia Tech", "Tony Bennett"],

    2425: ["loss daughter", "daughter feels", "life will never", "without misty"],

    4242: ["new york", "Childhood friends", "former neighbor", "without paying",
        "dagan said", "jail"],

    16273: ["arbitration proceeding", "last week", "sanders said"],

    2909: ["nancy pelosi", "white house", "american", "embraced inner", "inner statesman",
        "His administration reveled", "Navy SEAL", "rave reviews", "wednesday morning"],

    #_#
    16066: ["background checks", "sporting goods"],
    16899: ["National Rifle", "Rifle Association", "Second Amendment",
     "repealing amendment", "extremely difficult", "two thirds", "background checks"],

    17664: ["lawyer Sven", "Sven Mar", "30 days", "Islamic State", "mary said"],
    16127: ["officials spoke"],
    #16931: [],
    17858: ["maintained his innocence", "power", "additional accusers", "civil suit",
        "five women", "referred to prison", "$3.4 million"],

    # TOMORROW
    16942: ["child protective", "protective services", "Dana DeKalb", "missing children", "DeKalb said"],

    16274: ["Gwala said", "Jackson said", "Police said", "South Africa", "roads"],#meta: south africa
    16944: ["million", "seven", "day", "time", "episode"],

    16394: ["peace deal", "Sergio Fajardo", "Humbero de", "de la", "la calle", "million votes",
        "peace accord","Rodrigo Londono", "mining exports"],

    17091: ["Perez Rivera", "Mujica said", "filing for asylum", "daughters",
        "single muncipal", "muncipal police", "police officer", "numbers"],

    18027: ["Brjawi", " reformulated electoral", "electoral law", "proportional representation",
        "polling stations", "Streets were festooned", "percent"],#meta: polling stations, percent

    17413: ["African American", "American male", "implicit bias", "legal obligation",
        "Kenny said", "Lauren Wimmer", "two black", "black men"],


    17576: ["$500 million", "Wells Fargo", "Federal Reserve", "Bank regulatros", "abusive practices",
        "CFPB", "Time Sloan"],#meta: Wells Fargo, CFPB

    16775: ["Africa", "asia", "europe and central", "central asia", "extinct",
        "Watson", "percent", "world"], # meta:Watson, africa, world, europe,asia, percent
    17602: ["vegetable", "contaminated vegetables",
        "officals have not found", "not found origin",
        "toss any romaine", ""],

    "17348": ["computer screen", "television", "soon", "expand", "happy and so excited",
        "boutique", "firm dates", "presence", ""],

    "17238": ["68", "year", "old", "former Republican", "democratic"],

    "16832": ["confetti-covered", "halftime", "Wolverines beat", "Florida State", "58-54"],

    "16170": ["start tweeting", "Mainstream Media", "mocked", "gone CRAZY!", "early start",
        "florida before his late", "late afternoon"],

    "16635": ["New York", "York City", "City Council", "Council members", "investigation",
    "contends initiated", "initiated by previous" "previous landord","Kushner Cos"],

    "17982": ["allegiance", "loyalty", "israeli", "iranian", "Israel", "iran"],

    "2424": ["reckless overprescribing", "overprescribing", "combat misuse", "abuse",
        "pharmaceutical industry", "industry group", "proposal burdensome", "burdensome arbitrary",
        "struggling with addiction"],

    "17133": ["panel", "adults who do not have landline", "landline phones", "20 percent"],#stopwords:who do not have

    "16112": ["pick lulu","yelled angrily","angrily into the microphone", "severe mental",
    "mental distress", "krim testified", "stared straight", "straight ahead", "dance class"],

    "17313": ["state transport", "transport taxes", "new york", "york city", "city yellow", "yellow cab",
    "attorneyclient privilege", "fbi agents", "Taxi King", "medallions", "$5 million"],


    "16687":  ["Trump son-in-law", "son-in-law Jared", "Jared Kushner", "White house", "envoy",
    "Jared Greenblatt","Mideast peace", "peace plan", " Crown Prince", "Middle East"],

    "17736": ["DNA match", "police officer", "Golden State", "State Killer", "DeAngelo",
    "emerging technology", "sacramento county", "suspect", "yelling and screaming",
    "district attorney"],


    "16726": ["major breach", "cnn interview", "SCL Group", "David Carroll",
    "sued Cambridge", "facebook post", "facebook officals"],

    "17970": ["drone footage", "summit magma", "magma reservoir", "457 meters",
    "magnitude 5.0","Geological Survey", "leilani estates", "new ground", "ground cracks"],

    "16226": ["statuete""lost angeles ", "angeles Police",
     "Facebook video", "photographer", "byrant", "terry byrant",
     "governors ball", "no one named", "named terry", "police said"],

    "8150": ["Facebook photos", "congressional baseball",
    " District of Columbia", "thursday", "ammunition",
    "veterans affairs", "affairs facility", "local police"],

    # MORE!
    1717797: ["lungs", "pancreatic cancer", "growths spread",
        "spread to her lung", "from cancer in another", "another organ",
        "10 years"],

    1911132: ["unemployment rate", "surfaces", "symptoms", "paid leave",
        "Consumer spending", "Jay Powell", "Federal Reserve", "hard surfaces"],

    1927808: ["July 21", "August 4"],

    1913110: ["social distancing", "unemployment rate", "surfaces", "hard surfaces",
        "regular exercise", "tested", "hand washing"], # 1911132: have an equal removed part

    1928371: ["12", "26", "	latest climate", "climate change", "change news"],


    1913793: ["updated september", "september 1", "1 2020"],

    1759668: [" Mr. Wernick", "prime minister", "attorney general",
        "general to be at loggerheads", "good idea", "pushing for a settlement"], # meta prime minister, attorney general

     1913829: ["states are reopening", "reopening bit", "bit by bit", "particles",
     "materials tested", "layered coffee", "coffee filters"],


    1784584: ["Mr. Gandhi", "Mr. modi", "parliament", "votes", "Hail modi!",
        "fake exit", "exit polls", "united states"],

    1914884: ["paid leave", "regular exercise", "social distancing", "hand washing",
            "political leaders", "masks"],



    1802254: ["Mr. Buttigieg", "Mr. Obama", "Silicon Valley", "fund-raising",
        "second quarter"],#meta:  second quarter, obama, Buttigieg

    # 1/6
    1914930: ["	august 24", "july 27"],

    1934915: ["email us", "us at onpolitics", "onpolitics nytimescom", "17", "6"],# already with proprocessing

    1812246: ["God bless him", "mr franken", "Senate vote", "extend lifetime", "lifetime compensation",
        "simple"], #meta: mr franken

    1843906: ["Dr Peebles", "dark matter", "universe", "dr Mayor", "dr queloz",
    "dark energy", "peebles", "mayor", "stars"],


    1852775: ["pacific gas", "air", "power cuts", "safety", "high winds",
        "three million", "million people", "without power"],


    1919840: ["united states", "states leading", "leading source", "source coronavirus",
    "coronavirus cases", "followed by italy", "italy then china"],

    # 2/6
    1892270: ["texas", "american airlines","airlines flight", "health officals",
        "flight attendants" ,"Dr Lofy"],


    1797395: ["Michael Savage", "Savage has doubts", "doubts about trump", "conservative audience",
            "eight"],

    1717936: ["New York", "inbox", "need to know", "start the day"],

    1757219: ["said", "maintained"],

    1718290: ["Norwegian immigrant", "Seussentennial", "Seussentenial"],

    1759523: ["Central American", "American governments", "government shutdown",
        "state department", "aid", "squandering"],

    # 3/6
    1801333: ["political", "sign up", "politcs", "world", "daily newsletter",
        "ideas shaping"],

    1761771: ["mail fraud", "potential tria", "Gallagher, who appeared", "months away",
        "firm"],

    1764919: ["virginia beach", "beach police", "virginian-pilot", "among hundreds",
    "hundreds of people", "Police Department"],

    1803047: ["census form", "citizenship question", "reiterated his unwillingness",
        "citizenship question", "give up in a twitter", "twitter message"],

    1722136: ["Linda Qiu", "southwest border", "Alan Rappeport", "mr trump",
        "billion", "border wall", "border security"],

    1777702: ["North Carolina", "law enforcement", "Mr Cane", "Mr Erickson",
        "gunmen"],

        # 4/6
        1806091: ["approximately", "five", "5"],
        1777809: ["legislative", "executive", "report", "mueller"],
        1778045: [" ballistic missile", "north korea", "united states",
             "time", "launched 4:30", "4:30 pm", "Strategic studies"],


        1809373: ["Justice Stevens", "district Columbia",
            "Columbia v", "suffered small", "small stroke"],

        1724172: ["surpass 5000", "5000 mark", "years later", ""],

        1781200: ["monday", "thursday", "blue origin", "rocket company",
            "started by Jeff", "jeff  Bezos", "starlink", "founder"],



        # 5/6
        1811196: ["train station", "hong kong", "homemade laboratory", "masked men",
            "police officers", "independent inquiry", "16-year-old"],#meta: hong kong

        1728138: ["American diplomats", "Trump administration", "Mr Maduro", "mr guaidó",
            "maximum mobilization", "first time"],

        1782393: ["Russia", "Chancellor Sebastian", "Sebastian Kurz", "freedom party",
            "european union", "snap election"],

        1812224: ["Sign up", "In Her Words", "newsletter", "women rule", "headlines",
            "newsletter"],

        1787172: ["government", "Kim", "north korean", "dictator", "remarks"],

        1814923: ["United Nations", "90 percent", "mr jawdat", " found suitable",
            "suitable work", "american officials", "camps", "releases"],

        # 6/6
        1750702: ["chief executive", "Max 8", "Mr Trump", "Boeing Max", "mr muilenburg",
            "flight attendants", "European Union", "737 Max"],



        1787638: ["seven people","seven confirmed", "confirmed dead",
            "33 South", "South Koreans", "deep condolences", "tour guide",
            "seven rescued"],


        1818974: ["violent"],

        1752632: ["Mr tarrent", "court papers", "Brenton Harrison", "harrison tarrant",
            "additional charges", "violence", "Christchurch courtroom", "gun laws"],

        1793437: ["pepper spray", "movement", "Hong Kong", "wore sunglasses", "surgical mask",
            "communist party"],#meta: Hong Kong

            1819591: ["police shot", "teenage demonstrator", "Oct. 1", "chief executive",
                "extradition bill", "	sept 4", "first time", "time in months",
                " live round"],



}

In [14]:
manual = {

    "90536": [
            "European Union", "cookies", "bugs",
            "personally identifiable", "Salesforce",
            "identifiable information", "customer testimonial",
            "version", "Third Party", "clear gifs", "web beacons",
            "service providers",  "blog"],


    "90344": ["Telerik", "May", "Progress Software", "software products", "software websites",
         "community progress", "Telerik Analytics", "please note", "Third party"
        "service providers", "geo-location",
         "google Analytics",  "children"],


    "98640": ["validation page", "scripts", "removing HTML5", "tools removing", "management tools",
         "various browsers", "clear gifs", "transparency", "may offer", "cookies", "flash cookies",
         "browsers may"],


    "98585": ["account settings", "center cloud", "cloud console", "security center", "console",
          "web beacons", "kaspersky securtiy", "personal data",
           "data", "email address", "solution kaspersky", "documentation section", "account registration"],


    "99880": ["reproductive science", "science center", "Michele Abbott",
    "electronic", "health information", "insurance company", "health", "shady grove",
    "treatment"], #meta: shady grove

    "108079": ["destination nsw","Insights", "Viva Sydney","Sydney Club", "130000 visitors","visitors per", "per month",
    "e-newsletter", "personal information", "please contact"],

    "90555":  ["serve ads", "DART cookie", "registration number", "privacy policy", "data control", "data protection"],


    "90545":  ["Yahoo privacy", "Privacy Policy", "AOL", "Oath", "AOL registred","registred users" ,"Verizon","oath privacy"],

    "98553": ["customer" "affiliated websites", "providers", "billing information",
    "contact us", "service plans", "partners", "advertising partners", "march 8", "april 20"],


    "98572": ["job applications", "Privacy Shield", "Shield Framework",
          "secure data", "data recovery", "recovery services" "personal data"],

    "98706": ["Australian privacy", "post", "contact us", "telephone", "SEEK Learning", "SEEK commerical"
        "Head office"],


    "108052":  ["Reloaded Games", "K2 network"],


    "108097": ["Website Security", "PKI solutions", "DigiCert Inc" "Symantec website",  "Global company",
            "data transfer","transfers", "Download privacy", "service providers", "contractors", "around the world",
            "privacy statement", "privacy policy"],


    100541: ["live", "browse history", "Synchrony Bank", "Synchrony", "GE money", "money bank", "GEMB",
         "Credit Card", "bank Credit", "Billing Act", "EVINE live", "online privacy"],

    108771: ["tap", "ALTERNATIVE PRESS", "alternative", "press llc"],

    2435: ["properties", "sites", "requests", "california", "545", "5215", "freeway suite",
        "john carpenter" ,"700", "1400", "Connor", "Blvd", "Connor Blvd"],

    100595: ["evine", "shopnbc", "ge money", "synchrony", "mobile", "mobile device", "money bank", "Synchrony Bank",
         "california", "credit card", "evine card", "ShopNBC platinum", "ShopNBC platinum rewards", "MasterCard"],

    108778: ["DART", "DART cookie", "parties", "third parties", "email", "log file", "file information",
        "personally identifiable"], # email is written differently

    100419: ["toxic shock", "tv", "shock tv" "advertising companies", "advertising", "third parties", "DART cookie",
         "web site", "serve ads", "web beacons", "cookie", "data collection"],



    108438: ["TakeLessons Live", "TakeLessons", "Released Subject", "Subject matter",
     "profile", "personal information", "current version", "update", "privacy policy",
     "unsubscribe"],

    108835: ["personally identifiable", " identifiable information", "cookies", "use", "privacy statement",
    "persitent cookies", "credit card", "support@tarot.com", "site"],#meta: site

    # small change examples
    106348: ["advertising", "www", "third party", "ad network", "strayer university"],
    90041: ["certain services", "mobile app", "app", "pages listed"],

# BEW
  89460: ["children", "player privacy", "privacy policy", "european union",
          "commerical partner", "personal information", "review"], # personal information (on a meta level)

  89884: ["debit card", "details", "credit debit", "SecureTrading", "order products", "competion terms",
      "conditions"],

  106296: ["Family mobile", "walmart", "geolocation", "phone" "device", "california",
         "network", "track", "data", "personal information"],

  89510: ["google adsense", "third party", "party advertisers","cookie", "dart cookie",
   "advertisers", "advertising partners", "partners include"],

  97991: ["google analytics", "version", "google", "GDPR",
        "screenpresso application", "amazon web", "web service", "Ip address"],

  89035: ["notice", "policy" "device", "automatically", "location information",
      "personally identifiable", "identifiable information", "service providers",
      "CareerBuilder", "Adobe Flash"], # advertising on a meta level

  98022: ["employees and contractors","Audio/Video Transcription", "Transcription Service",
         "required sign", "sign non", "non disclousure", "disclousure agreement"],

  # ~15 Min break: return at 14:35

  89139: ["location information", "device", "personally identifiable", "identifiable information",
   "automatically collect", "third parties", "service providers", "Last updated", "opt out"],

  89648: ["can-spam compliance",  "marketing act", "securtiy measures", "nonsolicited pornography",
        "security measures", "can-spam compliance", "controlling assault"],

  97653 : ["secure", "personal information", "mobile", "data", "technical", "opt", "opt out", "improve"],

  98103: ["swingers", "free trial", "customer service", "lifesytle website", "travel", "local swingers",
         "find local", "membership", "lifestyle events"], # SDC on a meta level

  89170: ["cookies", "utilize cookies", "RTM websites", "third-party", "web browser",
   "advertising", "ad servers", "detailed information", "ip address", "data collection"],

  89763: ["cookies", "com", "mobile devices", "safe harbor", "email address"
      "google",  "advertising", "priceline", "please click"], # Meta Level: data, cookies, advertising

  97732: ["ScienceForums.net staff", "policy", "notice", "servers", "log files",
            "IP addresses", "web servers", "explicit consent", "action"],

  98116: ["LookupRealty", "SDLookup"],

  89273: ["healthkit database", "planned parenthood", "Donor Services" "parenthood National",
    "national office", "Apple's Healthkit", "ppfa", "online services"],# Meta: parenthood, planned parenthood

  89768: ["pricena fz",  "third parties", "right holder", "rights reserved", "updated", "pricena.com acts"], # meta level: pricena

  97787: ["dataprivacy aip.org","practices", "advertising based", "peer review", "personal information", "aip publishing"], # meta: personal information

  89306: ["dart cookie", "serve ads", "power point", "cookie", "third party", "vendor", "ppst", "google"],

  89860: ["cookies","primo website", "privacy statement", "revisions", "cookies disabled", "disabled flagged",
  "recognize browser"],

  97799: ["Digital offering", "personal data", "disclose", "international privacy", "privacy policy", "rights",
    "web sites", "US privacy"], # Meta: Digital offering

  # BREAK: 17:00

  # Restart: 19:00

  # TODO: 13 <- Mi 19:30
  104276: ["ad network", "advertising", "network", "network provider", "advertising notice"
          "advertisements", "site", "sending duplicative", "duplicative advertisements", "web beacons",
          ],

  80227: ["nokta Internet", "personal data", "data", "personal", "data processing",
        "protection act", "processed", "act", "Flash Player"],

  96434: ["CustomerNation", "privacy practices", "customer Service",
        "tom clinkscales", "web site", "address"],

  96759: ["gdpr", "New Zealand", "Swiss", "personal information",
      "personal information", "sessions clicks",  "visits sessions",
      "third party", "credit card", "google analytics", "user analytics"],

  104303: ["third-party vendors", "internet", "questions", "cookies", "opt out",
            "visitors", "Remarketing", "vendors", "SPC", "visiting", "website tracking",
            "information share"], # META: spc


  96461: ["IP addresses", "collect", "Herald & Times",  "Newsquest Herald",
          "personally identifiable", "identifiable information", "web beacons",
          "aggregate", "CV Database", "nonpersonally identifiable",  "email client"],# meta: collect, information, aggregate, identifiable

  # Finish 20:15


  #TODO (DO): 27 #NOTE: Done 16
  # START 10:30 <- left overs
  96970: ["email", "cookies", "use cookies", "privacy", "personal information",
        "information", "third party", "thirdparty partner", "children privacy",
        "email policy"], # meta: personal information, information

  104311: ["Github", "ordered list", "orderedlist", "support",  "inc"],

  96180: ["Australian", "disclose", "consent", "information", "members", "sponsors"], #skip

  96505: ["safe harbor", "shield framework", "privacy shield", "environmental variables",
        "Eu", "HTML5", "single sign", "swiss safe", "third party", "personal data"
        "facebook connect", "saba software"],#Meta: cookies

  96971: ["www", "secure shopping", "credit card", "personal information",
        "privacy policy", "cookies", "information", "children privacy"],#meta: www, information

  #31161: ["outside", "United states", "telemarketing", "Switzerland",
 #        "california", "transaction", "nevada", "applicable law", "protected"], # meta:applicable law

  # Finish: 11:00
  #NOTE: 10 min break

    #NOTE: Start 11:11
    96240: ["daily beast", "beast implemented", "recaptcha technology", "google invisible", "invisible recaptcha", "www", "http",
      "Nielsen",], #Meta: www, http

    96506: ["social media", "Widgets", "media widgets", "media Features",
        "ip address", "contact us", "third party"],

    97065: ["TRUSTe consent", "consent manager", "Google", "analytics", "Google analytics", "user id",
        "Adobe target", "Ip addresses", "sap group", "Adboe analytics", "Russian Federation"],#meta: SAP, user

    31168: ["Security", "opens", "new window", "collected information", "preferences", "Introduction",
    "customers", "fca US"],# site-buttons: opens, new windows # headers:fca, fca us

    96325: ["data", "rights", "data", "email protected",
    "general data", "data protection", "protection regulation", "European union"],#meta processing, data

    96665: ["personal information", "cookies", "browser", "credit card", "cutomers", "operate stores",
         "affiliated businesses", "Web sites", "please click", "SAFRA"],

  96346: ["google analytics", "pii", "web browser", "browser", "privacy policy", "analytics privacy"],


  80219: ["Siemens", "Solutions", "account", "google", "email address", "use cookies",
        "google analytics", "nokia Solutions", "site content", "enforce"],

  96414: ["ryan homes", "remains strictly", "strictly anonymous", "third party", "homes may",
         "information received", "internal statistical", "statistical purposes",
         "partner"],#meta: ryan homes # mentioned more often: strictly anonymous

  96711: ["privacy policy", "sainsbury bank", "adobe acrobat", "contact us",
         "top", "send us", "email", "explicit consent", "privacy statement"], # meta: privacy policy

  # #NOTE: 14:40
  102943: ["DotNetNuke", "DNN CORP"],

  104225: ["privacy policy", "google analytics", "device identifiers", "mobile device",
      "com", "advertiser feature", "analytics demographics", "identifiable information"],
      #meta: privacy policy, identifiable information

  94843: ["advertising", "third party", "services", "interest based",
      "behavioral advertising", "tracking analytics", "opt out"], # meta: third party, services,

  95640: ["Roland Berger", "countries outside", "eu eea", "third parties", "use data",
      "data protection", "Berger Strategy"], #Meta: Roland Berger, Berger Strategy

  103341: ["sensitive information", "Virtual terminal", "user", "Customer service",
      "take precautions", "online transactions",],

  104249: ["particle", "spark"],

  94977: ["Tigrent Learning", "Legacy Education", "education alliance", "alliance inc",
  "tel 2395420643", "Trowsers Road", "Parkshot House"],

  103581: ["SolarWinds Website", "statement", "privacy statement", "third party", "use cookies"], # meta: SolarWinds Website

  104254: ["Sparked", "Extraordinaries", "customers", "volunteer user", "phishing", "personal data",
  "client"],

  95008: ["fraud prevention", "prevention agencies", "details", "personal information",
    "illegal transactions", "failed attempts"],

  # Break

  95819: ["donation service", "advertising", "identifiable information", "internal businesses",
      "id address", "AppChoices"],

  103865: ["back top", "top", "Sony centre", "Sony styles", "third parties",
    "record information", "3ps", "style"], # meta: top/back

  104257: ["google analytics", "social media", "third party", "credit card", "email address",
      "service", "sparkfun electronics", "law enforcement", "60 days", "servers", "track"],

  104259: ["privacy shield", "standard contractual", "contractual clauses", "shark hire",
        "Google Adwords", "third parties", "Data protection", "data processing", "data",
        "agreement", "Controller"], # meta: shark hire, data


  95866: ["CTI", "PII", "web beacons", "cookies", "third parties", "parents",
      "surveys", "Other Entities", "account", "credit card"],


    103892: ["sca websites", "sony electronics", "sca", "SEL", "new york",
        "third party", "Device", "promotions", "service providers", "Sony drive"],
  # BREAK 10 more


  #95323: ["time warner", "warner cable", "spectrum", "tracking tools", ""],

  95888: ["anonymous data", "back top", "Rovi property", "data", "personal data",
       "public content", "Europe", "Social Networking", "Delaware", "data protection", "england"],

  103927: ["Columbia Sportswear", "Sportswear Company", "Safe Harbor", "harbor framework", "united states",
            "Switzerland", "Swiss"],


  104127: ["Tsogo sun", "information", "personal information", "personally identifiable",
            "identifiable information", "google analytics", "cookies", "Display advertising"], # meta: Tsogo Sun

  94812: ["third party", "web beacons", "social media", "media features",
        "Ip address", "Pixel tags", "tracking technologies", "may use"], #meta: tracking technologies

  95516: ["personal information", "non-personal information", "Rockford fosgate",
       "may collect", "information", "Ip address"],

  104129: ["children", "from outside", "united states", "third parties", "tailored advertising",
      "privacy setting", "may receive", "receive tailored"],

  94836: ["web sites", "revenue science", "testimonials", "personally identifiable",
      "identifiable information", "information practices"], #meta: web sites, revenue science

   102375: ["Boulder brands", "Smart Balance", "cosumermail", "webmaster"],

   4072: ["functionality cookies", "performance cookies"],
   4293: ["cookies", "additional information", "third party", "google analytics",
            "use cookies", "information", "website", "User Experience"],

   102421: ["safe harbor", "enterprise", "enterprise users", "enterprise level",
      "Harbor framework", "harbor privacy", "privacy principles", "Customer Service",
      "Test Question", "profile page", "profile information"],

   102542: ["Mercedes Me", "owners online", "such cases", "cases may", "may employ","employ outside",
    "outside research", "research company", "company call", "call behalf"], # 1 new sentence <- keywords in 2-grams

   4278: ["social security", "link", "external link", "session cookies", "Air force",
            "monitoring"], # meta: air force
  # 6 MORE:

    93866: ["RedZee", "Link Positions", "search results", "identify"],

    93873: ["personally identifiable", "identifiable information", "affiliated companies", "real estate",
        "vendors", "broker"],

    93932: ["ReferralSaaSquatch", "Yupiq", "personal data", "data", "personal information", "com"],

    94137: ["phone - call", "call tollfree", "back top",  "personal information", "non-personal", "products and services"],

    93880: ["reed jewelers", "sites", "comentity"],

    94002 : ["Social media", "widgets", "persitent", "web site", "third party", "session id",
    "media features", "Facebook Connect"]
}

## Compare Against Manually Annotated Data

In [11]:
import string
import nltk

def remove_punctuation(text, symbols=string.punctuation):
    return "".join([char for char in text if char not in symbols])

stop_words = nltk.corpus.stopwords.words("english")

In [12]:
def process_text(string):
    string = remove_punctuation(string)
    words_to_use = []
    for word in nltk.word_tokenize(string):
        if word not in stop_words:
            words_to_use.append(word)
        
    return " ".join(words_to_use)

In [15]:
manual = {site: [process_text(word) for word in words] for site, words in manual.items()}
news = {site: [process_text(word) for word in words] for site, words in news.items()}

In [16]:
def evaluate_manual(manual, compare="pipeline", typ="manual", path = "Dataframes"):
    f1s = []
    precs = []
    recalls = []
    overlaps = []
    IoUs = []
    num_manual = []
    stemmer = nltk.stem.PorterStemmer()
    sites = list(manual.keys())
    
    for site in sites:
        
        kws = manual[site]
        
        kws_hat = list(pd.read_csv(f"{path}/{typ}_{compare}_keywords_{site}.csv").keyword)
        
        kws = [stemmer.stem(kw) for kw in kws]

        kws_hat = [stemmer.stem(str(kw_hat)) for kw_hat in kws_hat][:max(len(kws), 10)]
        
        overlap = summary.number_of_overlaps(set(kws), set(kws_hat))
        
        precision = overlap / max(1, len(kws_hat))

        recall = overlap / max(1, len(kws))

        f1 =  2 * (precision * recall) / max(1, precision + recall)
        IoU = summary.intersection_over_union(kws, kws_hat)

        f1s.append(f1)
        precs.append(precision)
        recalls.append(recall)
        overlaps.append(overlap)
        num_manual.append(len(kws))
        IoUs.append(IoU)
    summary_frame = pd.DataFrame({'site_id': [int(site) for site in sites], 
                                  'f1': f1s, 'precision': precs,
                                  'recall': recalls, 'IoU': IoUs, '#overlaps': overlaps,
                                 '#manual keywords': num_manual})
    
    return summary_frame
    

In [17]:
display(evaluate_manual(news, compare="pipeline", typ=f"standard").drop("site_id", axis=1).describe())

,f1,precision,recall,IoU,#overlaps,#manual keywords
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.458177,0.418597,0.611522,0.360734,3.750000,6.250000
std,0.275776,0.242990,0.307260,0.240447,2.016973,2.124294
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.257143,0.300000,0.421429,0.200000,3.000000,5.000000
50%,0.533333,0.400000,0.666667,0.363636,4.000000,6.000000
75%,0.666667,0.511364,0.857143,0.500000,5.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,8.000000,11.000000


In [ ]:
def show_keywords_combinations(combinations, prefix):
  for i in range(len(combinations)):
    print(f"Results for combination {i}: {combinations[i]}")

    print("Pipeline Results: ")
    display(evaluate_manual(manual, compare="pipeline", typ=f"{prefix}_{i}"))

In [31]:
def describe_combinations(manual, combinations, prefix):
  for i in range(len(combinations)):
    print(f"Results for combination {i}: {combinations[i]}")

    print("Pipeline Results: ")
    pipe = evaluate_manual(manual, compare="pipeline", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()


    base1 = evaluate_manual(manual, compare="baseline1", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    base2 = evaluate_manual(manual, compare="baseline2", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    base3 = evaluate_manual(manual, compare="baseline3", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    base4 = evaluate_manual(manual, compare="baseline4", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    display(pd.DataFrame({"pipeline":pipe, "baseline1": base1, "baseline2": base2, "baseline3": base3, "baseline4": base4}).transpose())

## Results

In [32]:
describe_combinations(news, combinations, "combination")

Results for combination 0: (<function match_sentences_semantic_search at 0x7fb93ff22ee0>, <function text_rank_importance at 0x7fba935a20d0>, 0.5)
Pipeline Results: 


,f1,precision,recall,IoU,#overlaps,#manual keywords
pipeline,0.460812,0.416597,0.618939,0.358210,3.73,6.25
baseline1,0.033786,0.047881,0.066274,0.032877,0.40,6.25
baseline2,0.031343,0.080246,0.033929,0.026130,0.22,6.25
baseline3,0.120932,0.139536,0.185036,0.118231,0.95,6.25
baseline4,0.022683,0.046000,0.068175,0.030292,0.46,6.25


Results for combination 1: (<function match_sentences_semantic_search at 0x7fb93ff22ee0>, <function text_rank_importance at 0x7fba935a20d0>, 0.6)
Pipeline Results: 


,f1,precision,recall,IoU,#overlaps,#manual keywords
pipeline,0.456889,0.415597,0.615550,0.358949,3.74,6.25
baseline1,0.034774,0.048881,0.071214,0.033781,0.41,6.25
baseline2,0.027521,0.068694,0.028762,0.022616,0.18,6.25
baseline3,0.120688,0.138695,0.182290,0.117575,0.94,6.25
baseline4,0.022683,0.046000,0.068175,0.030292,0.46,6.25


Results for combination 2: (<function match_sentences_semantic_search at 0x7fb93ff22ee0>, <function text_rank_importance at 0x7fba935a20d0>, 0.7)
Pipeline Results: 


...

,f1,precision,recall,IoU,#overlaps,#manual keywords
pipeline,0.409846,0.377931,0.561256,0.320818,3.42,6.25
baseline1,0.032997,0.042881,0.062329,0.030007,0.35,6.25
baseline2,0.025886,0.062917,0.021024,0.018754,0.12,6.25
baseline3,0.111295,0.122695,0.151869,0.106497,0.79,6.25
baseline4,0.022683,0.046000,0.068175,0.030292,0.46,6.25


Results for combination 6: (<function match_sentences_tfidf_weighted at 0x7fb93ff22e50>, <function text_rank_importance at 0x7fba935a20d0>, 0.7)
Pipeline Results: 


,f1,precision,recall,IoU,#overlaps,#manual keywords
pipeline,0.371351,0.348788,0.516022,0.292731,3.15,6.25
baseline1,0.025000,0.037881,0.043841,0.023633,0.26,6.25
baseline2,0.027195,0.067500,0.021024,0.020040,0.12,6.25
baseline3,0.094209,0.110302,0.125746,0.090112,0.61,6.25
baseline4,0.022683,0.046000,0.068175,0.030292,0.46,6.25


Results for combination 7: (<function match_sentences_tfidf_weighted at 0x7fb93ff22e50>, <function text_rank_importance at 0x7fba935a20d0>, 0.8)
Pipeline Results: 


,f1,precision,recall,IoU,#overlaps,#manual keywords
pipeline,0.339020,0.319970,0.477898,0.267670,2.88,6.25
baseline1,0.025634,0.035131,0.042290,0.022275,0.23,6.25
baseline2,0.015052,0.047500,0.010929,0.010361,0.07,6.25
baseline3,0.084061,0.106854,0.108088,0.081913,0.50,6.25
baseline4,0.022683,0.046000,0.068175,0.030292,0.46,6.25
